In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Thu Mar 16 23:08:52 2017

@author: JunTaniguchi
"""
import os
import selectivesearch
import cv2
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from keras.optimizers import Adam
import numpy as np
from PIL import Image

os.chdir('/Users/JunTaniguchi/study_tensorflow/keras_project/read_place/project_rcnn')

from vgg_model import vgg_model

#地名のリストを作成
with open("./param/place_tokyo.txt", "r") as place_file:
    place_list = place_file.readlines()
    place_list = [place_str.strip() for place_str in place_list]
NUM_CLASSES = len(place_list)

# モデルの作成
def schedule(epoch, decay=0.9):
    return base_lr * decay**(epoch)
base_lr = 3e-4
optim = Adam(lr=base_lr)
input_shape = (200, 200, 3)

#実際に確認したいターゲットの画像を設定する。
img = cv2.imread("hrei-sign105.png")

model = vgg_model(input_shape, NUM_CLASSES, optim) 

def predict_img(img, model, place_list):
    # 画像から特徴部分を抽出
    img_lbl, regions = selectivesearch.selective_search(img,
                                                        scale=500,
                                                        sigma=0.9,
                                                        min_size=10)
    # 分類器を使用して判別
    candidates = set()
    for r in regions:
        # excluding same rectangle (with different segments)
        if r['rect'] in candidates:
            continue
        # excluding regions smaller than 2000 pixels
        if r['size'] < 10:
            continue
        # distorted rects
        x, y, w, h = r['rect']
    #    if w / h > 1.2 or h / w > 1.2:
    #        continue
        candidates.add(r['rect'])
    
    # draw rectangles on the original image
    fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(6, 6))
    ax.imshow(img)
    
    # 
    for x, y, w, h in candidates:
        #print (x, y, w, h)
        rect = mpatches.Rectangle(
            (x, y), w, h, fill=False, edgecolor='red', linewidth=1)
        ax.add_patch(rect)
    plt.show()
    
    img_list = []
    result_list = []
    for x, y, w, h in candidates:
        img_part = img[y:y+h, x:x+w]
        image = Image.fromarray(img_part)
        image.show()
        img_resize = cv2.resize(img_part, (200, 200))
        print(img_resize.shape)
        img_list.append(img_resize)
        
    result_list = model.predict(img_list)
    for idx, result_idx in enumerate(result_list):
        prime_result_idx = result_idx[idx].argmax()
        if prime_result_idx > 0.9:
            part_place_name = place_list[prime_result_idx]
            result_list.append(part_place_name)
    
    return result_list

result_list = predict_img(img, model, place_list)
print(result_list)